# Inicjalizacja kontrolera

In [1]:
import can
import matplotlib.pyplot as plt
from matplotlib import interactive
import numpy as np
import os
import time
import math
from IPython.display import display, clear_output
from joint_can import *
%matplotlib widget

In [2]:
def calibrate_joint(cmd,bus):
    send_msg(0x0A0, None, bus)
    time.sleep(2)
    
    if (send_msg(0x010, None, bus).data[0] == 1):
        send_msg(0x010, [cmd]*6, bus)
        time.sleep(1)
        return True
    else:
        return False
        # print(send_msg(0x010, [3], bus).data[0])
        # time.sleep(5)        

# Parametry kalibracji

In [3]:
serial="JANUSZ_4_RI70_21209057" #musi byc min jedna litera w numerze
note="joint_J4_pcb_3_4" #póki co musi być cokolwiek

In [4]:
path = "/home/pi/mech_diagnostics/diagnostic_data"
try: 
    os.mkdir(path+'/'+serial) 
except OSError as error: 
    pass

In [5]:
os.popen('/home/pi/mech_diagnostics/can_down.sh')
time.sleep(1)
os.popen('/home/pi/mech_diagnostics/can_up.sh')
time.sleep(1)


bus = can.interface.Bus(bustype='socketcan', channel='can0', bitrate=1000000, data_bitrate=5000000, fd=True)

# Reset adresów CAN

In [6]:
send_msg(0x030, [64,1,0,0,0,0,0,0], bus)
time.sleep(0.1)
send_msg(0x040, None, bus)
time.sleep(0.1)
send_msg(0x0a0, None, bus)
time.sleep(1)

# Kalibracja enkodera absolutnego

In [7]:
while not calibrate_joint(131,bus):
    time.sleep(1)
    
while(get_data(send_msg(0x000,None,bus),bus).FSM != 1):


    print(f'Calibration in progress\r', end='')
    time.sleep(1)
print('                              \r',end='')

send_msg(0x010, [2], bus)
time.sleep(0.1)
send_msg(0x010, [1], bus)
time.sleep(0.1)
send_msg(0x040, None, bus)
time.sleep(1)

print(f'1/2 Calibration done', end='')

1/2 Calibration done          

# Kalibracja zera

In [ ]:
while not calibrate_joint(161,bus):
    time.sleep(1)
    
while(get_data(send_msg(0x000,None,bus),bus).FSM != 1):


    print(f'Calibration in progress\r', end='')
    time.sleep(1)
print('                              \r',end='')

send_msg(0x010, [2], bus)
time.sleep(0.1)
send_msg(0x010, [1], bus)
time.sleep(0.1)
send_msg(0x040, None, bus)
time.sleep(1)

print(f'2/2 Calibration done', end='')

# Test enkoderów

In [ ]:
# do sprawdzenia enkodera nie uruchamiać bloków sekcji 1
send_msg(0x0A0, None, bus)
time.sleep(1)
send_msg(0x0F0, [2,2], bus)
send_msg(0x010, [2], bus)
time.sleep(0.1)
send_msg(0x010, [3], bus)
time.sleep(6)

In [ ]:
position_data=[]
speed_data=[]

set_speed(0.1,bus)
time.sleep(1)

run_time=60
start_time=time.time()

while((time.time()-start_time)<(run_time)):
    data=set_speed(0.1,bus)
    position_data.append(data.position)
    speed_data.append(data.speed)
set_speed(0.,bus)

In [ ]:
plt.plot(position_data)
# plt.plot(speed_data)
plt.suptitle(serial+ '\n' +note)
plt.show()
plt.savefig(path+'/'+serial+'/enkoder_'+time.strftime('%y_%m_%d_%H_%M')+'.png', dpi=150)

# ustawienie na zerze [uruchomić komórki 1.1-1.5]

In [ ]:
send_msg(0x0A0, None, bus)
time.sleep(1)
send_msg(0x0F0, [2,2], bus)
send_msg(0x010, [2], bus)
time.sleep(0.1)
send_msg(0x010, [3], bus)
time.sleep(6)


data=set_speed(0,bus)

sgn=(int(data.position>0) - int(data.position<0))

data=set_speed(sgn*0.1,bus)

while(abs(data.position)>0.01):
    if (abs(data.position)>0.2):
        data=set_speed(sgn*-0.2,bus)
    else:
        data=set_speed(sgn*-0.05,bus)
    time.sleep(0.1)

send_msg(0x010, [2], bus)
print(data.position)